In [ ]:
import os

%load_ext autoreload
%autoreload 2

""""
Devo code for moment by moment decoding --> i.e., for each time bin, find strength of representation of
each class of a variable (e.g., shape), as opposed to decoding average accuracy across trials.


Here collecting methods and quick analyses
"""

### A note on how shapes are represented in chars

In [ ]:
# SUMMARY:
# seqc_{i}_shapesem and seqc_{i}_shapesemgrp are same between char and SP/PIG.
# seqc_{i}_shape is NOT the same between SP/PIG (it uses ground-truth shape labels) and char (it uses charclust labels).

# SP/PIG (it uses ground-truth shape labels) and char (it uses charclust labels), therefore matches on semantic

# EXAMPLES:
# (1) PIG/SP tasks during char day
# ---> 

# A SP trial where the charclust label (top) differs from the ground-truth label (bottom)
# ============== beh_using_beh_data
# shape  --  ['Lcentered-4-1-0']
# shape_semantic  --  ['Lcentered-UL-UL']
# gridloc  --  [(1, 1)]
# seqc_{i}_shape: ['zigzagSq-92-3.5-0.5']
# seqc_{i}_shapesem: ['zigzagSq-UU-0.0']
# seqc_{i}_shapesemgrp: ['ZZ-UU-0.0']
# ============== beh_using_task_data
# shape  --  ['zigzagSq-92-3.5-0.5']
# shape_semantic  --  ['zigzagSq-UU-0.0']
# gridloc  --  [(1, 1)]
# seqc_{i}_shape: ['zigzagSq-92-3.5-0.5']
# seqc_{i}_shapesem: ['zigzagSq-UU-0.0']
# seqc_{i}_shapesemgrp: ['ZZ-UU-0.0']

# A CHAR trial
# ============== beh_using_beh_data
# shape  --  ['line-8-2-0', 'squiggle3-3-1-1', 'arcdeep-4-3-0', 'line-8-1-0']
# shape_semantic  --  ['line-UU-UU', 'squiggle3-UU-0.0', 'arcdeep-RR-RR', 'line-LL-LL']
# gridloc  --  [('IGN', 'IGN'), ('IGN', 'IGN'), ('IGN', 'IGN'), ('IGN', 'IGN')]
# seqc_{i}_shape: ['line-8-2-0', 'squiggle3-3-1-1', 'arcdeep-4-3-0', 'line-8-1-0']
# seqc_{i}_shapesem: ['line-UU-UU', 'squiggle3-UU-0.0', 'arcdeep-RR-RR', 'line-LL-LL']
# seqc_{i}_shapesemgrp: ['line-UU-UU', 'ZZ-UU-0.0', 'ARC-RR', 'line-LL-LL']
# ============== beh_using_task_data
# shape  --  ['line-8-2-0', 'squiggle3-3-1-1', 'arcdeep-4-3-0', 'line-8-1-0']
# shape_semantic  --  ['line-UU-UU', 'squiggle3-UU-0.0', 'arcdeep-RR-RR', 'line-LL-LL']
# gridloc  --  [('IGN', 'IGN'), ('IGN', 'IGN'), ('IGN', 'IGN'), ('IGN', 'IGN')]
# seqc_{i}_shape: ['line-8-2-0', 'squiggle3-3-1-1', 'arcdeep-4-3-0', 'line-8-1-0']
# seqc_{i}_shapesem: ['line-UU-UU', 'squiggle3-UU-0.0', 'arcdeep-RR-RR', 'line-LL-LL']
# seqc_{i}_shapesemgrp: ['line-UU-UU', 'ZZ-UU-0.0', 'ARC-RR', 'line-LL-LL']


# Load a dataset

To load and plot a dataset of neural activity across population, in a PopAnal class object.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:
# Method 1 - load a single DFallPA
DFallpa = load_handsaved_wrapper()


In [ ]:
# Method 1 - load a single DFallPA
DFallpa = load_handsaved_wrapper(animal="Pancho", date=230623, version="trial")


In [ ]:
# Method 2 - Combine two dfallpa
animal = "Diego"
date = 231130
COMBINE_AREAS = True

DFallpa1 = load_handsaved_wrapper(animal=animal, date=date, version="trial", combine_areas=COMBINE_AREAS, use_time=True)
DFallpa2 = load_handsaved_wrapper(animal=animal, date=date, version="stroke", combine_areas=COMBINE_AREAS, use_time=True)
DFallpa = pd.concat([DFallpa1, DFallpa2]).reset_index(drop=True)


In [ ]:
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]
dflab["seqc_0_shapesemgrp"].value_counts()

# Sanity checks...

In [ ]:
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import dfallpa_preprocess_condition
dfallpa_preprocess_condition(DFallpa)

# Preprocessing

In [ ]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:

# Prune to chans that are common across pa for each bregion (intersection of chans)|
dfpa_match_chans_across_pa_each_bregion(DFallpa)

In [ ]:
SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/decode_moment/CHAR/{animal}-{date}"
os.makedirs(SAVEDIR, exist_ok=True)

# Preprocessing

##### Devo -- removing noisy channels

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concatbregion_preprocess_clean_bad_channels
dfpa_concatbregion_preprocess_clean_bad_channels(DFallpa, PLOT=False)

# Sqrt normalization

In [ ]:
for pa in DFallpa["pa"]:
    pa.X = pa.X**0.5

### FR Normalization

##### Method 1 - each PA normalize independently

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
plot_savedir = "/tmp"
dfallpa_preprocess_fr_normalization(DFallpa, fr_normalization_method, plot_savedir)

##### Method 2 - Concat events (for each bregion) and normalize all same way


In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
dfpa_concat_normalize_fr_split_multbregion(DFallpa)


##### Method 3 - concat events (flexible version, only constrianed to have same n chans across PA) [works if have both trial and stroke!]

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion_flex
fr_mean_subtract_method = "across_time_bins"
# fr_mean_subtract_method = "each_time_bin"
PLOT=False

pa = DFallpa["pa"].values[10]
pa.plotNeurHeat(0)

dfpa_concat_normalize_fr_split_multbregion_flex(DFallpa, fr_mean_subtract_method, PLOT)

pa = DFallpa["pa"].values[10]
pa.plotNeurHeat(0)

### Factorize all relevant labels FIRST here.


In [ ]:
if False: # Ingore this for now
    from neuralmonkey.analyses.decode_good import preprocess_factorize_class_labels_ints
    MAP_LABELS_TO_INT = preprocess_factorize_class_labels_ints(DFallpa)

# Extract relevant variables


In [ ]:
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import dfallpa_preprocess_condition
shape_var_suff = "shapesemgrp"
loc_var_suff = "loc_on_clust"
dfallpa_preprocess_condition(DFallpa, shape_var_suff, loc_var_suff)

# Train - test of decoder (on char)

In [ ]:
SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/decode_moment/CHAR/{animal}-{date}/v2"
os.makedirs(SAVEDIR, exist_ok=True)

In [ ]:
import numpy as np
for pa in DFallpa["pa"]:
    print("her")
    assert not np.any(np.isnan(pa.X))

In [ ]:
from neuralmonkey.analyses.decode_moment import analy_chars_score_postsamp, analy_chars_dfscores_condition


In [ ]:

analy_chars_score_postsamp(DFallpa, SAVEDIR) 

# DEVO

In [ ]:
# NOTE: all of these are in analy_chars_score_postsamp()

In [ ]:
from neuralmonkey.analyses.decode_moment import train_decoder_helper, pipeline_train_test_scalar_score, pipeline_train_test_scalar_score_mult_train_dataset, test_decoder_helper, train_decoder_helper_extract_train_dataset
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import get_dataset_params
import os

In [ ]:
bregion = "vlPFC"
SAVEDIR = "/tmp"
classifier_version = "logistic"
# classifier_version = "ensemble"

In [ ]:
# Hard coded params:
include_null_data = False
n_min_per_var = 3
subtract_baseline=False
subtract_baseline_twind=(-0.45, -0.05)
PLOT = True

### Test params
# - post-samp
# test_dataset = "char_samp_post"
# var_test = "seqc_1_shapesemgrp"

test_dataset = "char_samp_post"
var_test = "seqc_0_shapesemgrp"

######### TRAINING PARAMS
list_train_dataset = ["sp_samp", "pig_samp", "sp_pig_pre_stroke_all"]
list_var_train = ["seqc_0_shapesemgrp", "seqc_0_shapesemgrp", "shape_semantic_grp"]
save_suff = "|".join(list_train_dataset)

# list_train_dataset = ["sp_samp", "pig_samp"]
# list_var_train = ["seqc_0_shapesemgrp", "seqc_0_shapesemgrp"]
# save_suff = "|".join(list_train_dataset)

# Extract some params
list_bregion = DFallpa["bregion"].unique().tolist()
event_test, _, filterdict_test, list_twind_test, which_level_test = get_dataset_params(test_dataset)

# Other params
savedir = f"{SAVEDIR}/traindata={save_suff}-testdata={test_dataset}/{bregion}/decoder_training_mult"
os.makedirs(savedir, exist_ok=True)
print(savedir)

dfscores, Dc, PAtrain, PAtest = pipeline_train_test_scalar_score_mult_train_dataset(DFallpa, bregion, 
                                    list_train_dataset, list_var_train, 
                                    var_test, event_test, list_twind_test, filterdict_test, 
                                    which_level_test, savedir, include_null_data, 
                                    prune_labels_exist_in_train_and_test=True, PLOT=PLOT, n_min_per_var=n_min_per_var,
                                    subtract_baseline=subtract_baseline, subtract_baseline_twind=subtract_baseline_twind,
                                    classifier_version=classifier_version)

# APPEND info related to trials.
dflab = PAtest.Xlabels["trials"]
dfscores = analy_chars_dfscores_condition(dfscores, dflab)

# list_decoder_class_idx_in_shapes_drawn = []
# list_decoder_class_was_drawn = []
# list_decoder_class_was_seen = []
# list_decoder_class_was_first_drawn = []

# for _i, row in dfscores.iterrows():

#     decoder_class = row["decoder_class"]
#     pa_idx = row["pa_idx"]
#     trialcode = row["trialcode"]
#     epoch = row["epoch"]

#     shapes_drawn = dflab.iloc[pa_idx]["shapes_drawn"]
#     FEAT_num_strokes_beh = dflab.iloc[pa_idx]["FEAT_num_strokes_beh"]
#     # shapes_visible = dflab.iloc[pa_idx]["taskconfig_shp"]
#     if decoder_class in shapes_drawn:
#         decoder_class_idx_in_shapes_drawn = shapes_drawn.index(decoder_class)
#     else:
#         decoder_class_idx_in_shapes_drawn = -1
    
#     assert FEAT_num_strokes_beh==len(shapes_drawn)
#     assert decoder_class_idx_in_shapes_drawn<FEAT_num_strokes_beh
    
#     list_decoder_class_idx_in_shapes_drawn.append(decoder_class_idx_in_shapes_drawn)
#     list_decoder_class_was_drawn.append(decoder_class in shapes_drawn)
#     # list_decoder_class_was_seen.append(decoder_class in shapes_visible)
#     list_decoder_class_was_first_drawn.append(decoder_class == shapes_drawn[0])
    
# dfscores["decoder_class_idx_in_shapes_drawn"] = list_decoder_class_idx_in_shapes_drawn
# dfscores["decoder_class_was_drawn"] = list_decoder_class_was_drawn
# # dfscores["decoder_class_was_seen"] = list_decoder_class_was_seen
# dfscores["decoder_class_was_first_drawn"] = list_decoder_class_was_first_drawn

# dfscores["FEAT_num_strokes_beh"] = [dflab.iloc[pa_idx]["FEAT_num_strokes_beh"] for pa_idx in dfscores["pa_idx"]]
# dfscores["bregion"] = bregion


# # Normalize decode by subtracting mean within each decoder class
# from pythonlib.tools.pandastools import datamod_normalize_row_after_grouping_return_same_len_df
# dfscores, _, _ = datamod_normalize_row_after_grouping_return_same_len_df(dfscores, "decoder_class_was_drawn", 
#                                                                         ["decoder_class"], "score", False, True, True)



##### [Good] pull out chars that have multiple parses

In [ ]:
from neuralmonkey.analyses.decode_moment import _analy_chars_score_postsamp_same_image_diff_parse
_analy_chars_score_postsamp_same_image_diff_parse(dfscores, "/tmp")

### Plot timecourse

In [ ]:
# Only keep shapes that have decoders

In [ ]:
savedir

In [ ]:
from neuralmonkey.analyses.decode_moment import _analy_chars_score_postsamp_plot_timecourse
_analy_chars_score_postsamp_plot_timecourse(Dc, PAtest, savedir)

# Trial by trial correlation

In [ ]:
dfscores

### Quantify --> entropy over shapes at start and end of planning period [TODO]

# Split trials by similar or different image distance

In [ ]:
savedir = "/tmp/image_distances"
os.makedirs(savedir, exist_ok=True)

In [ ]:
from neuralmonkey.analyses.decode_moment import _analy_chars_score_postsamp_timecourse_splitby_image_distance
_analy_chars_score_postsamp_timecourse_splitby_image_distance(Dc, PAtest, animal, date, savedir, twind_test)

# Euclidian distances between pairs of trials

In [ ]:
from neuralmonkey.analyses.decode_moment import _analy_chars_score_postsamp_image_distance_neural_distance
savedir = "/tmp/eucl"
os.makedirs(savedir, exist_ok=True)
_analy_chars_score_postsamp_image_distance_neural_distance(PAtest, var_test, animal, date, savedir)

##### Method 2 -- datapt = char. [OBSOLETE]

In [ ]:
# PROBLEM -- need to compare cases matching first shape drawn -- or else PMv effects are dominated by that.
# Instead, just use pairs of trials, above

In [ ]:


# Datapt = char
res = []
for i, char1 in enumerate(list_char):
    for j, char2 in enumerate(list_char):
        if j>i:

            # Get neural distance
            dist_neural = Cl.index_find_dat_by_label((char1,), (char2,))

            # Get shape distance
            dist_shape = shape_distance_compute(dflab, char1, char2)

            # Get image distance
            dist_image = distance_mat[i, j]

            res.append({
                "dist_neural":dist_neural,
                "dist_shape":dist_shape,
                "dist_image":dist_image,
                "char1":char1,
                "char2":char2,
                "i_char":i,
                "j_char":j,
                "n_strokes_char1":map_char_to_n_drawn_strokes[char1],
                "n_strokes_char2":map_char_to_n_drawn_strokes[char2]
            })



In [ ]:
dfres = pd.DataFrame(res)
dfres

In [ ]:
dfres["n_strokes_char1_binned"] = pd.cut(dfres["n_strokes_char1"], 4)
dfres["n_strokes_char2_binned"] = pd.cut(dfres["n_strokes_char2"], 4)

dfres["dist_shape_binned"] = pd.cut(dfres["dist_shape"], 6)
dfres["dist_image_binned"] = pd.cut(dfres["dist_image"], 6)


In [ ]:
sns.catplot(data=dfres, x="dist_image_binned", y="dist_neural", hue="dist_shape_binned", kind="point", 
            col="n_strokes_char1_binned", row="n_strokes_char2_binned")

sns.catplot(data=dfres, x="dist_shape_binned", y="dist_neural", hue="dist_image_binned", kind="point", 
            col="n_strokes_char1_binned", row="n_strokes_char2_binned")

In [ ]:
dfres["dist_shape"].hist(bins=50)

In [ ]:
import seaborn as sns
sns.relplot(data=dfres, x="dist_shape", y="dist_neural", alpha=0.1, col="n_strokes_char1_binned", row="n_strokes_char2_binned", height=5)

In [ ]:
sns.displot(data=dfres, x="dist_shape", y="dist_neural", col="n_strokes_char1_binned", row="n_strokes_char2_binned", 
            height=5, kind="kde")

In [ ]:
# Extract slices of data with high and low image and shape distance
pd

In [ ]:
import seaborn as sns
sns.relplot(data=dfres, x="dist_image", y="dist_neural", alpha=0.1, col="n_strokes_char1_binned", row="n_strokes_char2_binned")

In [ ]:
sns.displot()

In [ ]:
Cl.rsa_dataextract_with_labels_as_flattened_df()

In [ ]:
fig, ax = plt.subplots()
ax.hist(Cl.Xinput.flatten(), bins=20)

In [ ]:
PAtest.dataextract_as_distance_matrix_clusters_flex()